# Будем обучать Bert из базового библиотеки Replay

### Модель будет обучаться на решение задачи прогноза следующего фильма каждого пользователя.
### В качестве основы был взят пример из гита RePlay

https://github.com/sb-ai-lab/RePlay/blob/main/examples/10_bert4rec_example.ipynb

## 0. Установка и импорты нужных пакетов, библиотек и данных

Устанавливаем нужные пакеты

In [3]:
!pip install transformers
!pip install torch
!pip install tqdm
!pip install replay-rec
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.8/196.8 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 15.6 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00
  Created wheel for fixed-install-nmslib: filename=fixed_install_nmslib-2.1.2-cp310-cp310-linux_x86_64.whl size=14023936 sha256=13a9940e5dad

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 51.5 MB/s eta 0:00:00


Импортируем библиотеки и фиксируем `seed`

In [4]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback,
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback,
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import Bert4Rec
from replay.models.nn.sequential.bert4rec import (
    Bert4RecPredictionDataset,
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionBatch,
    Bert4RecModel
)

import pandas as pd

import random
import numpy as np

def set_global_seed(seed: int) -> None:

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_global_seed(42)

import warnings
warnings.filterwarnings("ignore")

In [5]:
spark_session = get_spark_session()

In [6]:
data = pd.read_csv('events.csv')
users = pd.read_csv('user_features.csv')
items = pd.read_csv('item_features.csv')

## 1. Предобработка данных

Переобозначаем `timestamp`, не нарушая порядок: пользователь с одним и тем же timestamp-ом, вероятно, баг - поэтому для поддержания последовательности для модели необходимо переобозначение

In [7]:
data["timestamp"] = data["timestamp"].astype("int64")
data = data.sort_values(by="timestamp")
data["timestamp"] = data.groupby("user_id").cumcount()

После нескольких прогонов моделей с разными гиперпараметрами на основе валидационной выборки был выбран лучший набор, представленный в ячейках скрипта.</br></br>
Итоговую модель обучаем без val выборки.</br>
В качестве тестовой выборки берутся последние интеракции для каждого пользователя.

In [8]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(data)
raw_train_events, raw_train_gt = splitter.split(raw_test_events)

Функция для объединения в одну рабочую структуру данных информации об интерациях, пользователях и фильмах. Накидываем ее на тренировочную и тестовую выборки

In [9]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

In [10]:
train_dataset = Dataset(feature_schema=prepare_feature_schema(is_ground_truth=False),
                        interactions=raw_train_events,
                        query_features=users,
                        item_features=items,
                        check_consistency=True,
                        categorical_encoded=False)
test_dataset = Dataset(feature_schema=prepare_feature_schema(is_ground_truth=False),
                       interactions=raw_test_events,
                       query_features=users,
                       item_features=items,
                       check_consistency=True,
                       categorical_encoded=False)
test_gt = Dataset(feature_schema=prepare_feature_schema(is_ground_truth=True),
                  interactions=raw_test_gt,
                  check_consistency=True,
                  categorical_encoded=False)

Прописываем схему для соотнесения наших выборок с тензорным представлением для работы модели

In [11]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(name=ITEM_FEATURE_NAME,
                      is_seq=True,
                      feature_type=FeatureType.CATEGORICAL,
                      feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
                      feature_hint=FeatureHint.ITEM_ID,
                      embedding_dim=300) #тут пробовал варианты от 100 до 500
                            )

Используем `SequenceTokenizer` для токенизации данных в наших датасетах.</br>
- Токенизация учитывает последовательность интераций пользователя для построения токенов
- Учитывается внутренняя информация пользователей и айтемов
- Классическая BERT-овская токенизация с доп токенами и субтокенами `CLS`, `SEP` для последовательностей, а также `PAD`, `0` и `1` для `attention_mask`

In [12]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(test_dataset)
sequential_validation_gt = tokenizer.transform(test_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt)

test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

## 2. Создание объектов модели, трейнера и даталоадеров

В качестве основы модели выступает архитектура `BERT`:</br>
- Внутренний слой размера `hidden_size=300`
- Количество голов трансформера `head_count=4`
- Количество трансформерных блоков `block_count=2`
- Дропаут `dropout_rate=0.52`</br>
  При начальном обучении лучше всего себя показал вариант с `dropout_rate=0.5`. Для итогового предсказания, так как модель обучается уже на train+val, дропаут немного увеличим для контроля переобучения
- В качестве оптимизатора используем `FatOptimizerFactory` (под капотом `Adam`)

In [13]:
MAX_SEQ_LEN = 100
BATCH_SIZE = 128
NUM_WORKERS = 4

model = Bert4Rec(tensor_schema,
                 block_count=2,
                 head_count=4, # 4, 6, 8
                 max_seq_len=MAX_SEQ_LEN,
                 hidden_size=300, #100, 300, 500
                 dropout_rate=0.52, #0.3, 0.5, 0.7
                 optimizer_factory=FatOptimizerFactory(learning_rate=1e-3)) #1e-3, 1e-4

В качестве основной метрики для коллбеков используем `recall@10` как в итоговой метрике соревнования

In [14]:
checkpoint_callback = ModelCheckpoint(dirpath=".checkpoints",
                                      save_top_k=1,
                                      verbose=True,
                                      monitor="recall@10",
                                      mode="max")

validation_metrics_callback = ValidationMetricsCallback(metrics=["recall"],
                                                        ks=[10],
                                                        item_count=train_dataset.item_count,
                                                        postprocessors=[RemoveSeenItems(sequential_validation_dataset)])

csv_logger = CSVLogger(save_dir=".logs/train", name="RecommenderBERTModelv7")

trainer = L.Trainer(max_epochs=100,
                    callbacks=[checkpoint_callback, validation_metrics_callback],
                    logger=csv_logger)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
train_dataloader = DataLoader(
    dataset=Bert4RecTrainingDataset(sequential_train_dataset,
                                    max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True)

validation_dataloader = DataLoader(
    dataset=Bert4RecValidationDataset(sequential_validation_dataset,
                                      sequential_validation_gt,
                                      sequential_train_dataset,
                                      max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True)

## 3. Обучение модели

In [16]:
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name   | Type             | Params | Mode 
----------------------------------------------------
0 | _model | Bert4RecModel    | 4.4 M  | train
1 | _loss  | CrossEntropyLoss | 0      | train
----------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.702    Total estimated model params size (MB)
38        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type             | Params | Mode 
----------------------------------------------------
0 | _model | Bert4RecModel    | 4.4 M  | train
1 | _loss  | CrossEntropyLoss | 0      | train
----------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.702    Total estimated model params size (M

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 48: 'recall@10' reached 0.03990 (best 0.03990), saving model to '/content/.checkpoints/epoch=0-step=48.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 48: 'recall@10' reached 0.03990 (best 0.03990), saving model to '/content/.checkpoints/epoch=0-step=48.ckpt' as top 1


...

INFO: Epoch 98, global step 4752: 'recall@10' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 98, global step 4752: 'recall@10' was not in top 1


k             10
recall  0.192881



Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 99, global step 4800: 'recall@10' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 99, global step 4800: 'recall@10' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


k             10
recall  0.183775



In [17]:
best_model = Bert4Rec.load_from_checkpoint(checkpoint_callback.best_model_path)

In [18]:
torch.save(best_model.state_dict(), "./RecommenderBERTModelv7.pth")

## 4. Инференс модели

Перезапуск ячеек был вызван сначала получением топ100 скоров для последующего отправления в бустинг. В этом скрипте остается топ10 для формирования итоговых рекомендаций

In [25]:
prediction_dataloader = DataLoader(
    dataset=Bert4RecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True)

csv_logger = CSVLogger(save_dir=".logs/test", name="RecommenderBERTModelv7")

В качестве коллбека используем специальный для `PySpark DataFrame`

In [26]:
TOPK = [10]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(spark_session=spark_session,
                                                    top_k=max(TOPK),
                                                    query_column="user_id",
                                                    item_column="item_id",
                                                    rating_column="score",
                                                    postprocessors=postprocessors)


Прокидываем в инференс тестовую выборку (последнее наблюдение датафрейма)

In [27]:
trainer = L.Trainer(callbacks=[spark_prediction_callback],
                    logger=csv_logger,
                    inference_mode=True)

trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [28]:
recs = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [29]:
result = recs.toPandas()
result

,score,user_id,item_id
0,2.702602,3,94
1,2.635417,3,2947
2,7.894511,4,755
3,7.267799,4,566
4,7.451857,18,495
...,...,...,...
60395,-0.840430,6016,793
60396,5.580224,6023,1868
60397,5.453481,6023,3035
60398,8.361889,6027,470


In [30]:
# result.to_csv('bert_scores.csv', index=False, header=True)

## 5. Формируем итоговый submission

In [31]:
top_recs = (
    result.sort_values(['user_id', 'score'], ascending=[True, False])
    .groupby('user_id')
    .agg(top_items=('item_id', lambda x: ' '.join(map(str, x.head(10)))))
    .reset_index())

top_recs.columns = ['user_id', 'item_id']
top_recs['user_id'] = top_recs['user_id'].astype(int)
top_recs = top_recs.sort_values('user_id', ascending=True)
top_recs.to_csv('submit_The_boysV6.csv', index=False, header=True)

top_recs

,user_id,item_id
0,0,1422 2003 1101 2593 708 1287 2801 1323 1259 3411
1,1,232 1246 350 560 3101 2518 1813 1459 1686 2561
2,2,2774 234 1371 2643 1781 2311 3431 1560 2354 382
3,3,3365 2169 3390 94 1316 3562 2947 3272 3441 3153
4,4,1160 983 755 394 3002 2185 566 3667 3602 3087
...,...,...
6035,6035,3665 2800 674 2366 513 3646 1011 3668 3216 1340
6036,6036,1102 3142 2502 922 2664 1057 3059 1011 3646 2800
6037,6037,1102 2502 1134 1379 1439 2800 3059 3646 3475 12
6038,6038,1514 3278 75 2420 1762 3185 1100 700 776 84
